In [1]:
# INTERFACING BETWEEN pandas and MODEL CODE

import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
# convert a DataFrame to numpy array

data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
# converting numpy array to a DataFrame

df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

In [7]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
df3 = data.copy()

In [9]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [10]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [11]:
df3.values

# the result of non-homogenous DF data is ndarray of Python objects

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [12]:
# wish to use a subset of the columns?
# use loc indexing with values

model_cols = ['x0', 'x1']

In [13]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [14]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories=['a', 'b'])

In [15]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [16]:
dummies = pd.get_dummies(data.category, prefix='category')

In [17]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [18]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


In [25]:
# CREATING MODEL DESCRIPTIONS WITH Patsy

# Patsy adalah library Python untuk deskripsi model statistika dengan small string-based 'formula-syntax'
# inspired by R dan S programming language

# Patsy -> specifying linear models in statsmodels
# patsy formulas: y ~ x0 + x1

data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [26]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [27]:
import patsy

In [31]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [32]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [33]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [34]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [35]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [38]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

# Adding 0 will suppress the intercept

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [39]:
# Patsy object  can be passed into algorithms like numpy.linalg.lstsq (linear algebra - least square)
# this will perform ordinary least squares regression

coef, resid, _, _ = np.linalg.lstsq(X, y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  after removing the cwd from sys.path.


In [40]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [41]:
resid

array([19.63791494])

In [42]:
_

array([8.03737688, 3.38335321, 0.90895207])

In [46]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [47]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [48]:
# Data Transformations in Patsy Formulas

y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1)+1)', data)

In [49]:
X

# intercept adalah titik singgung di sumbu ___

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [51]:
data.x1

0    0.01
1   -0.01
2    0.25
3   -4.10
4    0.00
Name: x1, dtype: float64

In [52]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [53]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [54]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

In [55]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

# apply transformations to new out-of-sample data using the saved information from the original in-sample dataset

In [56]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [57]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [58]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [59]:
# Categorical Data and Patsy

# Non-numeric data can be transformed for a model design matrix in many different ways

data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [60]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [61]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [62]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [63]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [64]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [65]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [66]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [67]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [68]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [69]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [70]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [71]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [72]:
# Introduction to statsmodels

# for fitting many kinds of statistical models, performing statistical tests, data exploration and visualization

# Estimating Linear Models

# there are many kinds of linear regression models
# basic (Ordinary Least Squares) -> complex (Iteratively Reweighted Least Squares)

In [73]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# linear models in statsmodels -> array based and formula based interface

In [74]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [81]:
# For reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
         dnorm(0, 0.6, size=N),
         dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [82]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [83]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [84]:
train = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
test = 